In [1]:
import h5py
import os
import numpy as np
import tensorflow as tf
from sklearn.linear_model import Ridge
from pprint import pprint
import json
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

if not os.path.isdir('results/lorenz_grid_lfads_new_vs_old'):
    os.mkdir('results/lorenz_grid_lfads_new_vs_old')

In [2]:
grid_dir = './results/lorenz_grid_lfads_new_vs_old/results'
subdirs = list(os.walk(grid_dir))[0][1]
print('Directories:\n', '\n '.join(subdirs))

Directories:
 train_trials=100|baselinerate=10|behaviour_sigma=1.0
 train_trials=100|baselinerate=15|behaviour_sigma=1.0
 train_trials=50|baselinerate=5|behaviour_sigma=2.0
 train_trials=200|baselinerate=15|behaviour_sigma=1.0
 train_trials=100|baselinerate=10|behaviour_sigma=2.0
 train_trials=200|baselinerate=10|behaviour_sigma=2.0
 train_trials=50|baselinerate=10|behaviour_sigma=0.5
 train_trials=200|baselinerate=5|behaviour_sigma=2.0
 train_trials=200|baselinerate=10|behaviour_sigma=0.5
 train_trials=100|baselinerate=5|behaviour_sigma=1.0
 train_trials=50|baselinerate=10|behaviour_sigma=2.0
 train_trials=100|baselinerate=15|behaviour_sigma=2.0
 train_trials=100|baselinerate=15|behaviour_sigma=0.5
 train_trials=200|baselinerate=10|behaviour_sigma=1.0
 train_trials=200|baselinerate=5|behaviour_sigma=0.5
 train_trials=100|baselinerate=5|behaviour_sigma=0.5
 train_trials=100|baselinerate=5|behaviour_sigma=2.0
 train_trials=50|baselinerate=15|behaviour_sigma=0.5
 train_trials=200|baselin

## Old LFADS Implementation

In [3]:
datasets = [('train', 'train'), ('valid', 'validation'), ('test', 'test')]
old_stats = {}

for subdir in subdirs:
    subdir_stats = {}
    ground_truth_file = h5py.File(os.path.join(grid_dir,
                                  subdir,
                                  'dataset.h5'), 'r')
    ridge_model = None
    for dataset_old, dataset in datasets:
        filename = os.path.join(grid_dir,
                                      subdir, 
                                      'results_old', 
                                      'model_runs__%s_posterior_sample_and_average' % (
                                      dataset_old))
        if not os.path.isfile(filename):
            continue
        file = h5py.File(filename, 'r')
        subdir_stats[dataset] = {}
        subdir_stats[dataset]['neural_likelihood'] = np.mean(file['nll_bound_vaes'])
        factors = np.asarray(file['factors'])
        latent = np.asarray(ground_truth_file['%s_latent' % (dataset_old)])
    
        z_unsrt = factors.T.reshape(factors.T.shape[0], factors.T.shape[1] * factors.T.shape[2]).T
        l = latent.T.reshape(latent.T.shape[0], latent.T.shape[1] * latent.T.shape[2]).T
        if ridge_model is None:
            ridge_model = Ridge(alpha=1.0)
            ridge_model.fit(z_unsrt, l)
        z_srt = ridge_model.predict(z_unsrt)
        unexplained_error = tf.reduce_sum(tf.square(l - z_srt)).numpy()
        total_error = tf.reduce_sum(tf.square(l - tf.reduce_mean(l, axis=[0,1]))).numpy()
        l_r2 = 1 - (unexplained_error / (total_error + 1e-10))
        subdir_stats[dataset]['latent_r2'] = l_r2
    old_stats[subdir] = subdir_stats

old_stats = {k: v for k, v in old_stats.items() if v}
pprint(old_stats)

{'train_trials=100|baselinerate=10|behaviour_sigma=1.0': {'test': {'latent_r2': 0.881334133840644,
                                                                   'neural_likelihood': 1186.8332918701171},
                                                          'train': {'latent_r2': 0.9570005916059897,
                                                                    'neural_likelihood': 1161.9935009765625},
                                                          'validation': {'latent_r2': 0.8930675359436979,
                                                                         'neural_likelihood': 1169.861806640625}},
 'train_trials=100|baselinerate=10|behaviour_sigma=2.0': {'test': {'latent_r2': 0.8824828878780975,
                                                                   'neural_likelihood': 1190.2368182373048},
                                                          'train': {'latent_r2': 0.9596548173466684,
                                                  

## New LFADS Implementation

In [4]:
subdir

'train_trials=50|baselinerate=5|behaviour_sigma=1.0'

In [5]:
new_stats = {}
setup = {}

for subdir in subdirs:
    try:
        with open(os.path.join(grid_dir, subdir, 'results_lfads', 'performance.json'), 'r') as pfp:
            perf_data = json.load(pfp)
        new_stats[subdir] = {k: {vk: vv for vk, vv in v.items() if vv is not None} 
                             for k, v in perf_data.items()}
    except BaseException as e:
        pass

    with open(os.path.join(grid_dir, subdir, 'results_lfads', 'metadata.json'), 'r') as mfp:
        metadata = json.load(mfp)
    setup[subdir] = {k: v for k, v in metadata.items() if v is not None}
    
pprint(new_stats)

{'train_trials=100|baselinerate=10|behaviour_sigma=0.5': {'test': {'latent_r2': 0.9184358616756175,
                                                                   'neural_likelihood': 1178.2435},
                                                          'train': {'latent_r2': 0.9468965911015064,
                                                                    'neural_likelihood': 1171.0834375},
                                                          'validation': {'latent_r2': 0.9212446640867732,
                                                                         'neural_likelihood': 1172.021640625}},
 'train_trials=100|baselinerate=10|behaviour_sigma=1.0': {'test': {'latent_r2': 0.9174415205669474,
                                                                   'neural_likelihood': 1163.80125},
                                                          'train': {'latent_r2': 0.9495409036104827,
                                                                    'neural

## Plotting

In [6]:
new_df = pd.DataFrame([{'trials': setup[k]['dataset']['train_pct'],
  'baseline_rate': setup[k]['dataset']['base_rate'],
  'behaviour_sigma': setup[k]['dataset']['behaviour_sigma'],
  'train_r2': v['train']['latent_r2'], 
  'train_likelihood': v['train']['neural_likelihood'],
  'validation_r2': v['validation']['latent_r2'], 
  'validation_likelihood': v['validation']['neural_likelihood'], 
  'test_r2': v['test']['latent_r2'],
  'test_likelihood': v['test']['neural_likelihood']}
    for k, v in new_stats.items()], index=list(new_stats.keys()))
new_df.sort_values(['trials', 'baseline_rate', 'behaviour_sigma'], inplace=True)
new_df.to_csv('results/lorenz_grid_lfads_new_vs_old/new_results.csv')
new_df

,trials,baseline_rate,behaviour_sigma,train_r2,train_likelihood,validation_r2,validation_likelihood,test_r2,test_likelihood
train_trials=50|baselinerate=5|behaviour_sigma=0.5,50,5,0.5,0.884371,732.261094,0.794346,765.049922,0.812516,754.888000
train_trials=50|baselinerate=5|behaviour_sigma=1.0,50,5,1.0,0.854558,730.503125,0.801252,732.600937,0.806748,743.817437
train_trials=50|baselinerate=5|behaviour_sigma=2.0,50,5,2.0,0.906366,714.245781,0.864728,721.583047,0.840279,723.117562
train_trials=50|baselinerate=10|behaviour_sigma=0.5,50,10,0.5,0.928593,1181.796406,0.823487,1186.625313,0.845715,1199.906625
train_trials=50|baselinerate=10|behaviour_sigma=1.0,50,10,1.0,0.932170,1176.021250,0.815915,1209.189687,0.814592,1198.506875
train_trials=50|baselinerate=10|behaviour_sigma=2.0,50,10,2.0,0.937542,1163.552031,0.850786,1170.036875,0.851311,1187.304375
train_trials=50|baselinerate=15|behaviour_sigma=0.5,50,15,0.5,0.947462,1531.782500,0.827905,1525.916094,0.812208,1566.255875
train_trials=50|baselinerate=15|behaviour_sigma=1.0,50,15,1.0,0.947636,1531.882188,0.829197,1538.619688,0.811711,1583.046750
train_trials=50|baselinerate=15|behaviour_sigma=2.0,50,15,2.0,0.943742,1518.158750,0.830910,1551.939844,0.846056,1553.273250
train_trials=100|baselinerate=5|behaviour_sigma=0.5,100,5,0.5,0.918470,726.075312,0.905061,732.549375,0.900220,736.195000


In [7]:
old_df = pd.DataFrame([{'trials': setup[k]['dataset']['train_pct'],
  'baseline_rate': setup[k]['dataset']['base_rate'],
  'behaviour_sigma': setup[k]['dataset']['behaviour_sigma'],
  'train_r2': v['train']['latent_r2'], 
  'train_likelihood': v['train']['neural_likelihood'],
  'validation_r2': v['validation']['latent_r2'], 
  'validation_likelihood': v['validation']['neural_likelihood'], 
  'test_r2': v['test']['latent_r2'],
  'test_likelihood': v['test']['neural_likelihood']}
    for k, v in old_stats.items()], index=list(old_stats.keys()))
old_df.sort_values(['trials', 'baseline_rate', 'behaviour_sigma'], inplace=True)
old_df.to_csv('results/lorenz_grid_lfads_new_vs_old/old_results.csv')
old_df

,trials,baseline_rate,behaviour_sigma,train_r2,train_likelihood,validation_r2,validation_likelihood,test_r2,test_likelihood
train_trials=50|baselinerate=5|behaviour_sigma=0.5,50,5,0.5,0.600752,756.283400,0.519603,794.083838,0.536084,784.640045
train_trials=50|baselinerate=5|behaviour_sigma=1.0,50,5,1.0,0.682719,755.891711,0.586498,760.906305,0.623883,771.295839
train_trials=50|baselinerate=5|behaviour_sigma=2.0,50,5,2.0,0.848900,729.985544,0.639637,751.292266,0.639788,751.551899
train_trials=50|baselinerate=10|behaviour_sigma=0.5,50,10,0.5,0.937741,1197.256239,0.735314,1215.647977,0.782543,1228.052583
train_trials=50|baselinerate=10|behaviour_sigma=1.0,50,10,1.0,0.941816,1190.724954,0.793619,1237.436016,0.787115,1224.238563
train_trials=50|baselinerate=10|behaviour_sigma=2.0,50,10,2.0,0.941029,1176.376746,0.804719,1194.669631,0.818928,1211.037623
train_trials=50|baselinerate=15|behaviour_sigma=0.5,50,15,0.5,0.967734,1543.134338,0.867195,1538.069226,0.847703,1581.728096
train_trials=50|baselinerate=15|behaviour_sigma=1.0,50,15,1.0,0.970843,1537.497776,0.846869,1554.376360,0.834191,1595.724064
train_trials=50|baselinerate=15|behaviour_sigma=2.0,50,15,2.0,0.967922,1522.220684,0.803804,1585.804187,0.817768,1587.056580
train_trials=100|baselinerate=5|behaviour_sigma=0.5,100,5,0.5,0.690777,743.703757,0.653776,757.708912,0.654292,760.252500


In [27]:
col_switch_pct = 0.3

new_avg_r2 = pd.pivot_table(new_df, values='test_r2', index=['trials'],
                                    columns=['baseline_rate'], aggfunc=np.mean)
old_avg_r2 = pd.pivot_table(old_df, values='test_r2', index=['trials'],
                                    columns=['baseline_rate'], aggfunc=np.mean)

zmin=min([new_avg_r2.values.min(), old_avg_r2.values.min()])
zmax=max([new_avg_r2.values.max(), old_avg_r2.values.max()])
text_func=np.vectorize(lambda x: '%.1f' % x)
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.15)

fig.add_trace(
    go.Heatmap(
        z=old_avg_r2.values,
        x=['%d' % (x) for x in old_avg_r2.columns.tolist()],
        y=['%d' % (x) for x in old_avg_r2.index.tolist()],
        colorscale='Blues',
        zmin=zmin,
        zmax=zmax),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=new_avg_r2.values,
        x=['%d' % (x) for x in new_avg_r2.columns.tolist()],
        y=['%d' % (x) for x in new_avg_r2.index.tolist()],
        colorscale='Blues',
        zmin=zmin,
        zmax=zmax),
    row=1, col=2
)

col_switch = max(np.max(old_avg_r2.values), np.max(new_avg_r2.values)) * col_switch_pct + \
    min(np.min(old_avg_r2.values), np.min(new_avg_r2.values)) * (1-col_switch_pct)

fig.update_layout(
    title_text='Latent Reconstruction Performance', title_x=0.5,
    annotations=[
    dict(
        showarrow=False,
        x=iy,
        y=ix,
        xref='x1',
        font=dict(color='white' if 
                  (old_avg_r2.iloc[ix, iy]) > col_switch
                  else 'black', size=12),
        text='%.2f' % (old_avg_r2.iloc[ix, iy])) 
        for ix,iy in np.ndindex(old_avg_r2.values.shape)] + [
    dict(
        showarrow=False,
        x=iy,
        y=ix,
        xref='x2',
        font=dict(color='white' if 
                  (new_avg_r2.iloc[ix, iy]) > col_switch
                  else 'black', size=12),
        text='%.2f' % (new_avg_r2.iloc[ix, iy])) 
        for ix,iy in np.ndindex(new_avg_r2.values.shape)] +
    [
        dict(
            showarrow=False,
            xref='x domain',
            x=0.77,
            yref='y domain',
            y=1.08,
            text='Tensorflow 1 - Original'),
        dict(
            showarrow=False,
            xref='x domain',
            x=2.1,
            yref='y domain',
            y=1.08,
            text='Tensorflow 2 - New'),
    ])
fig['layout']['xaxis1'].update(title='Baseline Firing Rate [Hz]')
fig['layout']['xaxis2'].update(title='Baseline Firing Rate [Hz]')
fig['layout']['yaxis1'].update(title='Training Trials')
fig['layout']['yaxis2'].update(title='Training Trials')

fig_diff = make_subplots(rows=1, cols=1)

fig_diff.add_trace(
    go.Heatmap(
        z=new_avg_r2.values - old_avg_r2.values,
        colorscale='Blues',
        x=['%d' % (x) for x in new_avg_r2.columns.tolist()],
        y=['%d' % (x) for x in new_avg_r2.index.tolist()]),
    row=1, col=1
)

col_switch = np.max((new_avg_r2 - old_avg_r2).values) * col_switch_pct + \
    np.min((new_avg_r2 - old_avg_r2).values) * (1-col_switch_pct)
fig_diff.update_layout(
    title_text='Latent Reconstruction Difference', title_x=0.5,
    annotations=[
    dict(
        showarrow=False,
        x=iy,
        y=ix,
        xref='x1',
        font=dict(color='white' if 
                  (new_avg_r2.iloc[ix, iy] - old_avg_r2.iloc[ix, iy]) > col_switch
                  else 'black', size=12),
        text='%.2f' % (new_avg_r2.iloc[ix, iy] - old_avg_r2.iloc[ix, iy])) 
        for ix,iy in np.ndindex(old_avg_r2.values.shape)])
fig_diff['layout']['xaxis1'].update(title='Baseline Firing Rate [Hz]')
fig_diff['layout']['yaxis1'].update(title='Training Trials')

fig.show()
fig.write_image('results/lorenz_grid_lfads_new_vs_old/r2.pdf', width=700, height=440)

fig_diff.show()
fig_diff.write_image('results/lorenz_grid_lfads_new_vs_old/r2_diff.pdf', width=500, height=400)

In [9]:
new_avg_likelihood = pd.pivot_table(new_df, values='test_likelihood', index=['trials'],
                                    columns=['baseline_rate'], aggfunc=np.mean)
new_avg_likelihood.head()
old_avg_likelihood = pd.pivot_table(old_df, values='test_likelihood', index=['trials'],
                                    columns=['baseline_rate'], aggfunc=np.mean)

zmin=min([new_avg_likelihood.values.min(), old_avg_likelihood.values.min()])
zmax=max([new_avg_likelihood.values.max(), old_avg_likelihood.values.max()])
text_func=np.vectorize(lambda x: '%.1f' % x)
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Heatmap(
        z=old_avg_likelihood.values,
        x=['%d' % (x) for x in old_avg_likelihood.columns.tolist()],
        y=['%d' % (x) for x in old_avg_likelihood.index.tolist()],
        zmin=zmin,
        zmax=zmax),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=new_avg_likelihood.values,
        x=['%d' % (x) for x in new_avg_likelihood.columns.tolist()],
        y=['%d' % (x) for x in new_avg_likelihood.index.tolist()],
        zmin=zmin,
        zmax=zmax),
    row=1, col=2
)

col_switch = max(np.max(old_avg_likelihood.values), np.max(new_avg_likelihood.values)) * col_switch_pct + \
    min(np.min(old_avg_likelihood.values), np.min(new_avg_likelihood.values)) * (1-col_switch_pct)

fig.update_layout(
    title_text='Latent Reconstruction Performance', title_x=0.5,
    annotations=[
    dict(
        showarrow=False,
        x=iy,
        y=ix,
        xref='x1',
        font=dict(color='black' if 
                  (old_avg_likelihood.iloc[ix, iy]) > col_switch
                  else 'white', size=12),
        text='%.1f' % (old_avg_likelihood.iloc[ix, iy])) 
        for ix,iy in np.ndindex(old_avg_likelihood.values.shape)] + [
    dict(
        showarrow=False,
        x=iy,
        y=ix,
        xref='x2',
        font=dict(color='black' if 
                  (new_avg_likelihood.iloc[ix, iy]) > col_switch
                  else 'white', size=12),
        text='%.1f' % (new_avg_likelihood.iloc[ix, iy])) 
        for ix,iy in np.ndindex(new_avg_likelihood.values.shape)] +
    [
        dict(
            showarrow=False,
            xref='x domain',
            x=2.1,
            yref='y domain',
            y=-0.22,
            text='Average log-likelihood loss (Hp: Poisson) on a 1000 trials test set, for 3 independent model runs.'),
        dict(
            showarrow=False,
            xref='x domain',
            x=0.74,
            yref='y domain',
            y=1.08,
            text='Tensorflow 1 - Original'),
        dict(
            showarrow=False,
            xref='x domain',
            x=1.92,
            yref='y domain',
            y=1.08,
            text='Tensorflow 2 - New'),
    ])
fig['layout']['xaxis1'].update(title='Baseline Firing Rate [Hz]')
fig['layout']['xaxis2'].update(title='Baseline Firing Rate [Hz]')
fig['layout']['yaxis1'].update(title='Training Trials')
fig['layout']['yaxis2'].update(title='Training Trials')

fig_diff = make_subplots(rows=1, cols=1)

fig_diff.add_trace(
    go.Heatmap(
        z=new_avg_likelihood.values - old_avg_likelihood.values,
        x=['%d' % (x) for x in new_avg_likelihood.columns.tolist()],
        y=['%d' % (x) for x in new_avg_likelihood.index.tolist()]),
    row=1, col=1
)

col_switch = np.max((new_avg_likelihood - old_avg_likelihood).values) * col_switch_pct + \
    np.min((new_avg_likelihood - old_avg_likelihood).values) * (1-col_switch_pct)

fig_diff.update_layout(
    title_text='Latent Reconstruction Difference', title_x=0.5,
    annotations=[
    dict(
        showarrow=False,
        x=iy,
        y=ix,
        xref='x1',
        font=dict(color='black' if 
                  (new_avg_likelihood.iloc[ix, iy] - old_avg_likelihood.iloc[ix, iy]) > col_switch
                  else 'white', size=12),
        text='%.1f' % (new_avg_likelihood.iloc[ix, iy] - old_avg_likelihood.iloc[ix, iy])) 
        for ix,iy in np.ndindex(old_avg_likelihood.values.shape)] +
    [
        dict(
            showarrow=False,
            xref='x domain',
            x=0.5,
            yref='y domain',
            y=-0.22,
            text='Difference in average log-likelihood loss (Hp: Poisson) on a 1000 trials test set, for 3 independent model runs.'),
    ])
fig_diff['layout']['xaxis1'].update(title='Baseline Firing Rate [Hz]')
fig_diff['layout']['yaxis1'].update(title='Training Trials')

fig.show()
fig.write_image('results/lorenz_grid_lfads_new_vs_old/likelihood.pdf', width=900, height=540)

fig_diff.show()
fig_diff.write_image('results/lorenz_grid_lfads_new_vs_old/likelihood_diff.pdf', width=900, height=540)